<a href="https://colab.research.google.com/github/tanimuranaomichi/Information_System_Analysis/blob/master/Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing and calculate similarity

このノートの目標は自力で文書の類似度を計算できるようになること  
最終的にWikipediaのデータを用いて国の類似度を測り  
日本と似ている国を探す

In [57]:
# 必要なパッケージのインストール
!pip install nltk
!pip install gensim

In [58]:
import nltk
import numpy as np
import pandas as pd

In [59]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Calculate similarity

以下の三つの文を考える  
Doc A : "I like apples and a strawberries. I will buy an apple tomorrow. "  
Doc B : "I bought some apples and strawberries. I will eat an apple tomorrow."  
Doc C : "I play basketball every day. I like Michael Jordan."  

Doc AとDoc Bは似ていそうだが、Doc CはDoc AともDoc Bとも似ていなさそう  
これを類似度を計算することで確かめる

類似度の計算の仕方はいくつかある

- 集合ベースの類似度
  - Jaccard係数
  - Dice係数
  - Simpson係数
- ベクトルベースの類似度
  - ユークリッド距離
  - コサイン類似度


### 集合ベース

文書を単語の集合に変換する  
集合なので重複した単語は削除する  
前処理は今回はスキップする   

Doc A : "I like apples and a strawberries. I will buy an apple tomorrow. "  
Doc B : "I bought some apples and strawberries. I will eat an apple tomorrow."  
Doc C : "I play basketball every day. I like Michael Jordan."  
↓    
Set A : {'a', 'an', 'and', 'apple', 'apples', 'buy', 'i', 'like', 'strawberries', 'tomorrow', 'will'}  
Set B : {'an', 'and', 'apple', 'apples', 'bought', 'eat', 'i', 'some', 'strawberries', 'tomorrow', 'will'}  
Set C : {'basketball', 'day', 'every', 'i', 'jordan', 'like', 'michael', 'play'}  

この集合が文書の特徴を表していると考える  


#### Jaccard係数
Jaccard係数は二つの集合A,Bに対して定義される類似度である  
計算式は以下の通り

\begin{equation}
J(A,B)=\dfrac{|A\cap B|}{|A \cup B|}
\end{equation}

共通部分の割合が大きければその二つの文書は似ていると考える

In [60]:
def jaccard_similarity(set_a,set_b):
  # 積集合の要素数を計算
  num_intersection = len(set.intersection(set_a, set_b))
  # 和集合の要素数を計算
  num_union = len(set.union(set_a, set_b))
  #Jaccard係数を算出　空集合の時は1を出力
  try:
      return float(num_intersection) / num_union
  except ZeroDivisionError:
      return 1.0 

In [61]:
set_a = set(['a', 'an', 'and', 'apple', 'apples', 'buy', 'i', 'like', 'strawberries', 'tomorrow', 'will'])
set_b = set(['an', 'and', 'apple', 'apples', 'bought', 'eat', 'i', 'some', 'strawberries', 'tomorrow', 'will'])
set_c = set(['basketball', 'day', 'every', 'i', 'jordan', 'like', 'michael', 'play'])

print("jaccard(a, b) = ", jaccard_similarity(set_a, set_b)) #Jaccard係数を計算
print("jaccard(a, c) = ", jaccard_similarity(set_a, set_c))
print("jaccard(b, c) = ", jaccard_similarity(set_b, set_c))

jaccard(a, b) =  0.5714285714285714
jaccard(a, c) =  0.11764705882352941
jaccard(b, c) =  0.05555555555555555



nltkで実装されている  
定義と同じように計算を行うので、入力は集合  
距離になっているところには注意が必要

In [62]:
from nltk.metrics import jaccard_distance

set_a = set(['a', 'an', 'and', 'apple', 'apples', 'buy', 'i', 'like', 'strawberries', 'tomorrow', 'will'])
set_b = set(['an', 'and', 'apple', 'apples', 'bought', 'eat', 'i', 'some', 'strawberries', 'tomorrow', 'will'])
set_c = set(['basketball', 'day', 'every', 'i', 'jordan', 'like', 'michael', 'play'])

# Jaccard距離になっているので、類似度に変換するときは1から引く
print("jaccard(a, b) = ", 1 - jaccard_distance(set_a, set_b))
print("jaccard(a, c) = ", 1 - jaccard_distance(set_a, set_c))
print("jaccard(b, c) = ", 1 - jaccard_distance(set_b, set_c))


jaccard(a, b) =  0.5714285714285714
jaccard(a, c) =  0.11764705882352944
jaccard(b, c) =  0.05555555555555558


#### Sørensen-Dice係数

Jaccard係数では分母はの和集合であったため  
片方の集合がとても大きいと共通部分が大きくても係数の値が小さくなってしまうという問題がある  
Sørensen-Dice係数では、分母を二つの集合の大きさの平均をとることで、その影響を緩和している  

$
DSC(A,B) = \dfrac{|A\cap B|}{\dfrac{|A| + |B|}{2}} = \dfrac{2|A\cap B|}{|A| + |B|}
$

In [63]:
def dice_similarity(set_a, set_b):
  num_intersection =  len(set.intersection(set_a, set_b))
  sum_nums = len(set_a) + len(set_b)
  try:
    return 2 * num_intersection / sum_nums
  except ZeroDivisionError:
    return 1.0 

In [64]:
set_a = set(['a', 'an', 'and', 'apple', 'apples', 'buy', 'i', 'like', 'strawberries', 'tomorrow', 'will'])
set_b = set(['an', 'and', 'apple', 'apples', 'bought', 'eat', 'i', 'some', 'strawberries', 'tomorrow', 'will'])
set_c = set(['basketball', 'day', 'every', 'i', 'jordan', 'like', 'michael', 'play'])

print("dice(a, b) = ", dice_similarity(set_a, set_b))
print("dice(a, c) = ", dice_similarity(set_a, set_c))
print("dice(b, c) = ", dice_similarity(set_b, set_c))

dice(a, b) =  0.7272727272727273
dice(a, c) =  0.21052631578947367
dice(b, c) =  0.10526315789473684


#### Szymkiewicz-Simpson係数

差集合の要素数の影響を極限まで抑えたのがSzymkiewicz-Simpson係数    
$
overlap(𝐴,𝐵) = \dfrac{|A\cap B|}{\min(|A|, |B|)}
$



In [65]:
def simpson_similarity(list_a, list_b):
  num_intersection = len(set.intersection(set(list_a), set(list_b)))
  min_num = min(len(set(list_a)), len(set(list_b)))
  try:
    return num_intersection / min_num
  except ZeroDivisionError:
    if num_intersection == 0:
      return 1.0
    else:
      return 0

In [66]:
set_a = set(['a', 'an', 'and', 'apple', 'apples', 'buy', 'i', 'like', 'strawberries', 'tomorrow', 'will'])
set_b = set(['an', 'and', 'apple', 'apples', 'bought', 'eat', 'i', 'some', 'strawberries', 'tomorrow', 'will'])
set_c = set(['basketball', 'day', 'every', 'i', 'jordan', 'like', 'michael', 'play'])

print("simpson(a, b) = ", simpson_similarity(set_a, set_b)) 
print("simpson(a, c) = ", simpson_similarity(set_a, set_c)) 
print("simpson(b, c) = ", simpson_similarity(set_b, set_c)) 

simpson(a, b) =  0.7272727272727273
simpson(a, c) =  0.25
simpson(b, c) =  0.125


#### Exercise 1
色々な集合を作って集合ベース手法の比較をしよう

In [67]:
set_a = set(['a', 'an', 'and', 'apple', 'apples', 'buy', 'i', 'like', 'strawberries', 'tomorrow', 'will'])
set_b = set(['an', 'and', 'apple', 'apples', 'bought', 'eat', 'i', 'some', 'strawberries', 'tomorrow', 'will'])
set_c = set(['basketball', 'day', 'every', 'i', 'jordan', 'like', 'michael', 'play'])
set_d = set() # 大きめの集合を作って試してみよう

print("jaccard similarity:")
print(jaccard_similarity(set_d, set_a))
print(jaccard_similarity(set_d, set_b))
print(jaccard_similarity(set_d, set_c))

print("dice similarity:")
print(dice_similarity(set_d, set_a))
print(dice_similarity(set_d, set_b))
print(dice_similarity(set_d, set_c))

print("simpson similarity:")
print(simpson_similarity(set_d, set_a))
print(simpson_similarity(set_d, set_b))
print(simpson_similarity(set_d, set_c))

jaccard similarity:
0.0
0.0
0.0
dice similarity:
0.0
0.0
0.0
simpson similarity:
1.0
1.0
1.0


### ベクトルベース 


文書をベクトルとして表現し類似度を計算する  
ベクトル化の手法は色々あるが今回はBoW(Bag of Words)で説明する  

BoWは文をベクトルで表現する方法の一つ  
想定している単語の総数をNとすると、各次元が各単語に対応するN次元のベクトルを考える  
各次元の値はその単語が文書中で出た回数

例）  
Doc A : "I like apples and a strawberries. I will buy an apple tomorrow. "  
Doc B : "I bought some apples and strawberries. I will eat an apple tomorrow."  
Doc C : "I play basketball every day. I like Michael Jordan."  
↓  
全単語は19個で、各次元の値は以下の単語の個数に対応するBoWを考える  
['an', 'and', 'apple', 'apples', 'basketball', 'bought', 'buy', 'day', 'eat', 'every', 'i', 'jordan', 'like', 'michael', 'play', 'some', 'strawberries', 'tomorrow', 'will']  
↓  
BoW A : [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1]  
BoW B : [1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 1, 1]  
BoW C : [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0]  

このベクトルが文書の特徴を表していると考える


#### ユークリッド距離

各文書をベクトルで表すことが出来たので  
ユークリッド距離が計算できる  
この距離が小さければ似ていると考えることが出来る

\begin{equation}
d(v_1,v_2) =(\sum_{i=1}^n (v_{1i}-v_{2i})^2)^{\frac{1}{2}}
\end{equation}

In [68]:
def euclidean_distance(list_a, list_b):
  diff_vec = np.array(list_a) - np.array(list_b)
  return np.linalg.norm(diff_vec)

In [69]:
bow_a = [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1]  
bow_b = [1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 1, 1]  
bow_c = [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0]  

print("euclidean_distance(bow_a, bow_b) = ",euclidean_distance(bow_a, bow_b))
print("euclidean_distance(bow_a, bow_c) = ",euclidean_distance(bow_a, bow_c))
print("euclidean_distance(bow_b, bow_c) = ",euclidean_distance(bow_b, bow_c))

euclidean_distance(bow_a, bow_b) =  2.23606797749979
euclidean_distance(bow_a, bow_c) =  3.7416573867739413
euclidean_distance(bow_b, bow_c) =  4.123105625617661


#### ミンコフスキー距離

ユークリッド距離を一般化した距離
pの値を変えることで色々な距離を表現できる  

\begin{equation}
d(v_1,v_2) = (\sum_{i=1}^n |v_{1i}-v_{2i}|^p)^{\frac{1}{p}}
\end{equation}

#### Exercise 2
ミンコフスキー距離を計算するプログラムを書いて  
p=1,2,3で距離を計算してみよう

In [70]:
# np.linalg.normについて調べよう
def minkowski_distance(list_a, list_b, p):
  diff_vec = np.array(list_a) - np.array(list_b)
  return np.linalg.norm(diff_vec, ord=p)
  

In [71]:
# p=1
print(minkowski_distance(bow_a, bow_b, 1))
print(minkowski_distance(bow_a, bow_c, 1))
print(minkowski_distance(bow_b, bow_c, 1))

# p=2
print(minkowski_distance(bow_a, bow_b, 2))
print(minkowski_distance(bow_a, bow_c, 2))
print(minkowski_distance(bow_b, bow_c, 2))

# p=3
print(minkowski_distance(bow_a, bow_b, 3))
print(minkowski_distance(bow_a, bow_c, 3))
print(minkowski_distance(bow_b, bow_c, 3))


5.0
14.0
17.0
2.23606797749979
3.7416573867739413
4.123105625617661
1.7099759466766968
2.4101422641752297
2.571281590658235


#### コサイン類似度

ベクトルのなす角に着目して類似度を計算する  

\begin{equation}
similarity(A, B)=cos(\theta)=\dfrac{\sum_{i=1}^n A_iB_i}{{\sqrt A}{\sqrt B}}
\end{equation}


#### Exercise 3
コサイン類似度を計算するプログラムを書いて計算しよう

In [72]:
# numpy.array について調べよう
def cosine_similarity(list_a, list_b):
  # あとで消す
  inner_prod = np.array(list_a).dot(np.array(list_b))
  norm_a = np.linalg.norm(list_a)
  norm_b = np.linalg.norm(list_b)
  try:
      return inner_prod / (norm_a*norm_b)
  except ZeroDivisionError:
      return 1.0

In [73]:
bow_a = [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1]
bow_b = [1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 1, 1]
bow_c = [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 0]

print("cosine_similarity(bow_a, bow_b) = ",cosine_similarity(bow_a, bow_b))
print("cosine_similarity(bow_a, bow_c) = ",cosine_similarity(bow_a, bow_c))
print("cosine_similarity(bow_b, bow_c) = ",cosine_similarity(bow_b, bow_c))

cosine_similarity(bow_a, bow_b) =  0.8153742483272114
cosine_similarity(bow_a, bow_c) =  0.41812100500354543
cosine_similarity(bow_b, bow_c) =  0.3223291856101521


### 集合ベースとベクトルベースの比較

集合演算の方は一つ一つの文書が小さいデータに対して性能が高い  
文書がある程度大きくなるとベクトルベースの方が有用になる  
その代わり、語彙集合が大きくなり計算量が大きくなってしまう


### Exercise 4
短い文章のデータセットと長い文章のデータセットを自分で作り    
Jaccard係数とコサイン類似度を計算して比較してみよう

In [56]:
short_docs = []
long_docs = []

## 2. Preprocessing

集合間の共通部分やベクトル間の距離や角度で類似度を測ることが出来た  
集合やベクトルが文書の特徴を上手く表せていないと類似度が上手く測れない  
文書からどのように集合やベクトルを作るかがとても大事  
 
適切な前処理を行うことで特徴を捉えた類似度を測れるようになる    
後半はベクトル化に絞って練習していく  

1. Clearning
2. Tokenize
3. Stemming
4. Remove stop words
5. Vectorize

### 2-1. Clearning

上の例では綺麗な文章ばかり扱っていたが、実際はもっと汚い   
Webから取ってきたデータだとhtmlタグが残っていたり、変な記号が入っていたりする  



In [37]:
documents=["I like apples and a strawberries. I will buy an apple tomorrow @Fresco.",
           "I bought some apples and strawberries. I will eat an apple <b>tomorrow.</b>",
           "I play basketball every day. I like Michael Jordan (born February 17, 1963)."]

今は三つなので手動で消せるが  
大量のデータを扱うときには自動で綺麗にできないといけない  
綺麗にするプログラムを作る

#### Exercise 5

正規表現を使ってテキストを綺麗にするプログラムを書こう

参考: 正規表現 (https://uxmilk.jp/41416)

In [74]:
import re

def cleaning_text(text):
    # @の削除
    pattern1 = '@'
    text = re.sub(pattern1, '', text)    
    # <b>タグの削除
    pattern2 = # 
    text = re.sub(pattern2, '', text)    
    # ()内を削除
    pattern3 = #
    text = re.sub(pattern3, '', text)
    return text
  

for text in documents:
    print(cleaning_text(text))

SyntaxError: ignored

#### Option 1

以下のテキストを綺麗にするコードを書いてみよう


In [ ]:
text = '<p><b>Natural language processing</b> (<b>NLP</b>) is a subfield of <a href="/wiki/Computer_science" title="Computer science">computer science</a>, <a href="/wiki/Information_engineering_(field)" title="Information engineering (field)">information engineering</a>, and <a href="/wiki/Artificial_intelligence" title="Artificial intelligence">artificial intelligence</a> concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of <a href="/wiki/Natural_language" title="Natural language">natural language</a> data.</p>'


### 2-2. Tokenize

まだ文字列のままなので、単語ごとに区切る  
英語だと空白区切りでよいが日本語だと少し面倒  

In [ ]:
def tokenize_text(text):
  text = re.sub('[.,]', '', text)
  return text.split()

for text in documents:
  text = cleaning_text(text)
  print(tokenize_text(text))

### 2-3. Stemming, Lemmatize

同じ意味の単語でも異なる形をしていることがある  
それらを別の単語としてカウントするのは不自然  
小文字に変換した後  
StemmingやLemmatizeという処理で同じ形にする  
今回はLemmatizeのみ

In [ ]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def lemmatize_word(word):
    # make words lower  example: Python =>python
    word=word.lower()
    
    # lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
      return lemma

In [ ]:
for text in documents:
  text = cleaning_text(text)
  tokens = tokenize_text(text)
  print([lemmatize_word(word) for word in tokens])

strawberries→strawberryのように語を標準形に変換出来た

### 2-4. Remove stop words

a, theなどの文章に寄らず一般的に使われる冠詞、代名詞、前置詞などを使っても意味がない  
それらの単語はstop wordと呼ばれる  
nltkには専門家が定義したstop wordのリストがあるのでそれを使う  
必要に応じてstop wordは自分でカスタマイズするべき  

In [ ]:
#1 nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')
print(en_stop)

In [ ]:
def remove_stopwords(word, stopwordset):
  if word in stopwordset:
    return None
  else:
    return word

In [ ]:
for text in documents:
  text = cleaning_text(text)
  tokens = tokenize_text(text)
  tokens = [lemmatize_word(word) for word in tokens]
  print([remove_stopwords(word, en_stop) for word in tokens])

今回はこれだけで終わりにするが単語の削除はかなり重要  
出現頻度が極端に低い単語を削除したり、動詞と名詞に限定するなど色々ある

In [ ]:
def preprocessing_text(text):
  text = cleaning_text(text)
  tokens = tokenize_text(text)
  tokens = [lemmatize_word(word) for word in tokens]
  tokens = [remove_stopwords(word, en_stop) for word in tokens]
  tokens = [word for word in tokens if word is not None]
  return tokens


preprocessed_docs = [preprocessing_text(text) for text in documents]
preprocessed_docs

### 2-5. Vectorize




#### BoW(Bag of Words)


テキストを単語の出現回数のベクトルで表したもの  
人手で単語を数えたりするのは不可能なのでプログラムで処理を完結してしまおう

In [ ]:
def bow_vectorizer(docs):
  word2id = {}
  for doc in docs:
    for w in doc:
      if w not in word2id:
        word2id[w] = len(word2id)
        
  result_list = []
  for doc in docs:
    doc_vec = [0] * len(word2id)
    for w in doc:
      doc_vec[word2id[w]] += 1
    result_list.append(doc_vec)
  return result_list, word2id

In [ ]:
bow_vec, word2id = bow_vectorizer(preprocessed_docs)
print(bow_vec)

In [ ]:
word2id.items()

### TF-IDF(Term Frequency - Inverse Document Frequency)

BoWでは各単語の重みが同じだったが、単語によって重要度は変わる  
単語の重要度を考慮したのがTF-IDF  

TF(t, d) = ある単語(t)のある文書(d)における出現頻度  
IDF(t) = ある単語(t)が全文書集合(D)中にどれだけの文書で出現したかの逆数  

TF-IDF(t,d) = TF(t, d) * IDF(t)  

In [ ]:
def tfidf_vectorizer(docs):
  def tf(word2id, doc):
    term_counts = np.zeros(len(word2id))
    for term in word2id.keys():
      term_counts[word2id[term]] = doc.count(term)
    tf_values = list(map(lambda x: x/sum(term_counts), term_counts))
    return tf_values
  
  def idf(word2id, docs):
    idf = np.zeros(len(word2id))
    for term in word2id.keys():
      idf[word2id[term]] = np.log(len(docs) / sum([bool(term in doc) for doc in docs]))
    return idf
  
  word2id = {}
  for doc in docs:
    for w in doc:
      if w not in word2id:
        word2id[w] = len(word2id)
  
  return [[_tf*_idf for _tf, _idf in zip(tf(word2id, doc), idf(word2id, docs))] for doc in docs], word2id
  

In [ ]:
tfidf_vector, word2id = tfidf_vectorizer(preprocessed_docs)
print(tfidf_vector)
print(word2id.items())

### Exercise 6
BoWとTF-IDFでコサイン類似度をそれぞれ計算してみよう

### Option 2
scikit-learn, nltk gensimそれぞれにTF-IDFを計算する関数がある  
それぞれでTF-IDFを計算してみよう

In [ ]:
# scikit-learnのtfidf　あとで消す


In [ ]:
#nltk のtf-idf　あとで消す
collection = nltk.TextCollection(docs)
terms = list(set(collection))
nltk_vector = []
for doc in docs:
  tmp_vec = np.zeros(len(word2id))
  for term in word2id.keys():
    tmp_vec[word2id[term]] = collection.tf_idf(term, doc)
  nltk_vector.append(list(tmp_vec))
print(nltk_vector)

In [ ]:
#gensim tf-idf あとで消す
from gensim import corpora
from gensim import models

dictionary = corpora.Dictionary(docs)
print('===単語->idの変換辞書===')
print(dictionary.token2id)
print(word2id)

corpus = list(map(dictionary.doc2bow, docs))
test_model = models.TfidfModel(corpus)
corpus_tfidf = test_model[corpus]

print('===結果表示===')
gensim_vector = []
for doc in corpus_tfidf:
  tmp_vec = [0] * len(word2id)
  for word in doc:
    key = dictionary[word[0]]
    tmp_vec[word2id[key]] = word[1]
  gensim_vector.append(tmp_vec)

print(gensim_vector)

## Exercise 7

様々な国のWikipediaにおけるabstractを取り出したデータセットを用意した  
https://drive.google.com/open?id=1i7tekPQRKaAwg-ze3kv5IsufMW13LkLo  
このデータをダウンロードして使う  

Cosine類似度の計算を行い、Japanに似ている国Top5を表示してみよう  
前処理を自分なりに工夫すること  
注）類似度はあまり高くならなくても良い  

In [ ]:
df = pd.read_csv("./nlp_country.csv")
df

In [ ]:
df.iloc[0]["Abstract"]

In [ ]:
# 後で消す
def preprocessing_text(text):
  def cleaning_text(text):
    # @の削除
    pattern1 = '@|%'
    text = re.sub(pattern1, '', text)    
    pattern2 = '\[[0-9 ]*\]'
    text = re.sub(pattern2, '', text)    
    # <b>タグの削除
    pattern3 = '\([a-z ]*\)'
    text = re.sub(pattern3, '', text)    
    pattern4 = '[0-9]'
    text = re.sub(pattern4, '', text)
    return text
  
  def tokenize_text(text):
    text = re.sub('[.,]', '', text)
    return text.split()

  def lemmatize_word(word):
    # make words lower  example: Python =>python
    word=word.lower()
    
    # lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
      return lemma
    
  text = cleaning_text(text)
  tokens = tokenize_text(text)
  tokens = [lemmatize_word(word) for word in tokens]
  tokens = [remove_stopwords(word, en_stop) for word in tokens]
  tokens = [word for word in tokens if word is not None]
  return tokens
  
docs = df["Abstract"].values
pp_docs = [preprocessing_text(text) for text in docs]
tfidf_vector, word2id = tfidf_vectorizer(pp_docs)

In [ ]:
word2id.items()

In [ ]:
def calc_cosine(vector, vector_list):
  result = {}
  for i, x in enumerate(vector_list):
    result[i] = cosine_similarity(vector, vector_list[i])
    
  return result

print("tfidf")
res = calc_cosine(tfidf_vector[0],tfidf_vector)
res

In [ ]:
sorted(res.items(), key=lambda x:x[1],reverse=True)

## Option 3

### Word2Vec & Doc2Vec

Word2VecやDoc2Vecでは単語の意味を捉えられているかのような演算が出来る  
King - Man + Woman = Queen など  
詳細は講義スライドへ   

学習済みのword2vecがgithub( https://github.com/Kyubyong/wordvectors )に上がっているので  
日本と各国の類似度を計算してみよう  
足し算や引き算が出来るのでそれも試してみよう  

参考 : "BOKU"のITな日常 (https://arakan-pgm-ai.hatenablog.com/entry/2019/02/08/090000)  